In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
% matplotlib inline
plt.style.use('seaborn-whitegrid')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
train = pd.read_csv("train.csv")

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

In [ ]:
#Fare amount is negative and it doesn't seem to be realistic
#few longitude and lattitude entries are off
#maximum passanger count is 208 which looks odd

In [ ]:
train.dropna(how='any', axis=0, inplace = True)

In [ ]:
train.shape #drop 376 

In [ ]:
train = train[train.fare_amount >=0]

In [ ]:
train.shape # drop 2454

In [ ]:
train[train.passenger_count >7].shape

In [ ]:
train=train[train.passenger_count <=7]

In [ ]:
train.shape #drop 101

In [ ]:
train[train.passenger_count ==0].shape

In [ ]:
#Passanger count is incorrectly populated
#Taxi was not carrying any passanger, may be taxi was used for goods

In [ ]:
train=train[train.passenger_count >0]

In [ ]:
train.shape # drop 195k

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
test.shape

In [ ]:
test.describe()

In [ ]:
#store the minimum and maximum of the longitude and latitude from test data set and filter the train data set for those data points

In [ ]:
min(test.pickup_longitude.min(),test.dropoff_longitude.min()), \
max(test.pickup_longitude.max(),test.dropoff_longitude.max())

In [ ]:
min(test.pickup_latitude.min(),test.dropoff_latitude.min()), \
max(test.pickup_latitude.max(),test.dropoff_latitude.max())

In [ ]:
def select_within_test_boundary(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])

In [ ]:
BB = (-74.5, -72.8, 40.5, 41.7)
train = train[select_within_test_boundary(train, BB)]
train.shape # drop 1,169,992

In [ ]:
train.head() 

In [ ]:
train.tail()

In [ ]:
test.head() 

In [ ]:
test.tail() 

In [ ]:
# eda sample 2,000,000 obs

In [ ]:
train_sample = train.sample(n=2000000)

In [ ]:
train_sample.shape

In [ ]:
# Lets see the distribution of fare amount 
train_sample.fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

In [ ]:
# Lets see the distribution of fare amount less than 100
train_sample[train_sample.fare_amount <=100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

In [ ]:
#There are few points between 40 and 60 dollars which has slightly high frequency and that could be airport trips

In [ ]:
train_sample[train_sample.fare_amount >100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

In [ ]:
#some of them might be outliers or few of them might be long distance trip from/to airport

In [ ]:
train_sample.passenger_count.hist(bins=10, figsize = (16,8))
plt.xlabel("Passanger Count")
plt.ylabel("Frequency")

In [ ]:
plt.figure(figsize= (16,8))
sns.boxplot(x = train_sample.passenger_count, y = train.fare_amount)

In [ ]:
f, ax = plt.subplots(1,2,figsize = [10,5])
sns.countplot(train_sample["passenger_count"], ax=ax[0])
sns.countplot(test["passenger_count"], ax=ax[1])
ax[0].set_title("Train Set - Passenger Count")
ax[1].set_title("Test Set - Passenger Count")
plt.show()

In [ ]:
# feature engineering
# time
# distance

In [ ]:
def prepare_time_features(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.replace(" UTC", "")
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
    df['hour_of_day'] = df.pickup_datetime.dt.hour
    df['week'] = df.pickup_datetime.dt.week
    df['month'] = df.pickup_datetime.dt.month
    df["year"] = df.pickup_datetime.dt.year
    df['day_of_year'] = df.pickup_datetime.dt.dayofyear
    df['week_of_year'] = df.pickup_datetime.dt.weekofyear
    df["weekday"] = df.pickup_datetime.dt.weekday
    df["quarter"] = df.pickup_datetime.dt.quarter
    df["day_of_month"] = df.pickup_datetime.dt.day
    return df

In [ ]:
train = prepare_time_features(train)
test = prepare_time_features(test)

In [ ]:
train.shape

In [ ]:
test.shape # add 9 time features

In [ ]:
# calculate-distance-between-two-latitude-longitude-points-haversine-formula 
# Returns distance in miles
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [ ]:
train['distance_miles'] = distance(train.pickup_latitude, train.pickup_longitude, \
                                      train.dropoff_latitude, train.dropoff_longitude)

In [ ]:
test['distance_miles'] = distance(test.pickup_latitude, test.pickup_longitude, \
                                      test.dropoff_latitude, test.dropoff_longitude)

In [ ]:
train[(train['distance_miles']==0)&(train['fare_amount']==0)].shape

In [ ]:
train = train.drop(train[(train['distance_miles']==0)&(train['fare_amount']==0)].index, axis=0)
train.shape # drop 178 obs 

In [ ]:
# Calculating pickup and drop distance from all 3 airports of Air Ports
def transform(data):
    # Distances to nearby airports, 
    jfk = (-73.7781, 40.6413)
    ewr = (-74.1745, 40.6895)
    lgr = (-73.8740, 40.7769)

    data['pickup_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = distance(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = distance(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data

In [ ]:
train = transform(train)
test = transform(test)

In [ ]:
train.shape # add 7 distance features totally

In [ ]:
train[train['fare_amount'] < 2.5].shape

In [ ]:
#the base fare for any taxi in new york is 2.5 dollars, we will drop those cases

In [ ]:
train = train.drop(train[train['fare_amount'] < 2.5].index, axis=0)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
train.to_csv('train_model.csv', index=False)

In [ ]:
test.to_csv('test_model.csv', index=False)

In [ ]:
# model
# lightgbm
# xgboost
# random forest
# catboost

In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import gc
import os

In [ ]:
# two big dataset: sample 5,000,000

In [12]:
%%time
train=pd.read_csv('train_model.csv')

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.84 ms


In [5]:
%%time
train= train.sample(n=20000000)     #取sample

CPU times: user 28.9 s, sys: 1.46 s, total: 30.4 s
Wall time: 31.2 s


In [13]:
train.shape

(200, 24)

In [14]:
test=pd.read_csv('test_model.csv')

In [ ]:
feats = [f for f in train.columns if f not in ['key','pickup_datetime','fare_amount']]

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()
    
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats], train['fare_amount'])):
    dtrain = lgb.Dataset(data=train[feats].iloc[train_idx], 
                         label=train['fare_amount'].iloc[train_idx],
                         free_raw_data=False)
    dvalid = lgb.Dataset(data=train[feats].iloc[valid_idx],
                         label=train['fare_amount'].iloc[valid_idx],
                         free_raw_data=False)
    params = {'boosting_type': 'gbdt',
              'objective': 'regression',
              'metric':'rmse',
              'learning_rate': 0.03,
              'num_leaves': 30, 
              'max_depth': 7,  
              'min_child_samples': 70,  
              'max_bin': 300,  
              'subsample': 1.0,  
              'subsample_freq': 1,  
              'colsample_bytree': 0.9,  
              'min_split_gain': 0.5,
              'min_child_weight': 4,
              'reg_lambda':0.1,
              'reg_alpha': 0.1,
              'nthread': 5,
              'verbose': -1,}
    
    clf = lgb.train(params, 
                    dtrain, 
                    valid_sets=[dtrain, dvalid], 
                    valid_names=['train','valid'],
                    num_boost_round=10000,
                    early_stopping_rounds=125,
                    verbose_eval=500)

    oof_preds[valid_idx] = clf.predict(train[feats].iloc[valid_idx])
    sub_preds += clf.predict(test[feats]) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d rmse : %.6f' % (n_fold + 1,mean_squared_error(train['fare_amount'].iloc[valid_idx], oof_preds[valid_idx]) ** .5)) 
    del clf, dtrain, dvalid
    gc.collect()

print('Full rmse %.6f' % mean_squared_error(train['fare_amount'], oof_preds)**.5)
# Write submission file and plot feature importance
sub_df = test[['key']].copy()
sub_df['fare_amount'] = sub_preds
sub_df[['key', 'fare_amount']].to_csv('submission_lgb2_10m.csv', index= False)

In [ ]:
cols = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).index
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
plt.figure(figsize=(8, 10))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout
plt.show()

In [ ]:
# bayesian optimization

In [15]:
from bayes_opt import BayesianOptimization

In [16]:
feats = [f for f in train.columns if f not in ['key','pickup_datetime','fare_amount']]
X=train[feats]
y=train['fare_amount']

In [22]:
def bayes_parameter_opt_lgb(X, y, init_round=2, opt_round=8, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.03, output_process=True):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y,free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, colsample_bytree, subsample, max_depth, reg_lambda, reg_alpha, min_split_gain, min_child_weight, 
                min_child_sample, max_bin, subsample_freq):
        params = {'objective':'regression','boosting_type': 'gbdt','nthread': 4, 'verbose': -1,\
                  'num_boost_round': n_estimators, 'learning_rate':learning_rate, \
                  'early_stopping_round':125}
        params['subsample_freq']=int(round(subsample_freq))
        params['min_child_sample']=int(round(min_child_sample))
        params['max_bin']=int(round(max_bin))
        params["num_leaves"] = int(round(num_leaves))
        params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
        params['subsample'] = max(min(subsample, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['reg_lambda'] = max(reg_lambda, 0)
        params['reg_alpha'] = max(reg_alpha, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=False, verbose_eval=500, metrics=['rmse'])
        return -1.0 * np.mean(cv_result['rmse-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (20, 100),
                                            'colsample_bytree': (0.6, 1.0),
                                            'subsample': (0.6, 1.0),
                                            'max_depth': (-1, 8),
                                            'reg_lambda': (0, 1),
                                            'reg_alpha': (0, 1),
                                            'min_child_sample':(20,100),
                                            'max_bin':(180,500),
                                            'subsample_freq':(1,10),
                                            'min_split_gain': (0.1, 0.8),
                                            'min_child_weight': (3, 20)})
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    #return lgbBO.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(X, y, init_round=2, opt_round=8, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.03,output_process=True)

Initialization
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_bin |   max_depth |   min_child_sample |   min_child_weight |   min_split_gain |   num_leaves |   reg_alpha |   reg_lambda |   subsample |   subsample_freq | 
    1 | 00m00s |   -6.11794 |             0.9511 |  485.5402 |      5.4910 |            67.3755 |             5.4100 |           0.4140 |      58.3420 |      0.0814 |       0.4842 |      0.8544 |           6.9699 | 
    2 | 00m00s |   -6.17495 |             0.6865 |  413.0501 |     -0.0459 |            70.5620 |            14.4540 |           0.4466 |      96.2448 |      0.3746 |       0.8531 |      0.8198 |           7.7255 | 
Bayesian Optimization
----------------------------------------------------------------------------------------------------

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=3560)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()
    
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats], train['fare_amount'])):
    dtrain = lgb.Dataset(data=train[feats].iloc[train_idx], 
                         label=train['fare_amount'].iloc[train_idx],
                         free_raw_data=False)
    dvalid = lgb.Dataset(data=train[feats].iloc[valid_idx],
                         label=train['fare_amount'].iloc[valid_idx],
                         free_raw_data=False)
    params = {'boosting_type': 'gbdt',
              'objective': 'regression',
              'metric':'rmse',
              'learning_rate': 0.03,
              'num_leaves': 25, 
              'max_depth': 7,  
              'min_child_samples': 36,  
              'max_bin': 333,  
              'subsample': 0.9525,  
              'subsample_freq': 3,  
              'colsample_bytree': 0.6416,  
              'min_split_gain': 0.1421,
              'min_child_weight': 4.0567,
              'reg_lambda':0.3254,
              'reg_alpha': 0.6913,
              'nthread': 8,
              'verbose': -1,}
    
    clf = lgb.train(params, 
                    dtrain, 
                    valid_sets=[dtrain, dvalid], 
                    valid_names=['train','valid'],
                    num_boost_round=20000,
                    early_stopping_rounds=125,
                    verbose_eval=500)

    oof_preds[valid_idx] = clf.predict(train[feats].iloc[valid_idx])
    sub_preds += clf.predict(test[feats]) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d rmse : %.6f' % (n_fold + 1,mean_squared_error(train['fare_amount'].iloc[valid_idx], oof_preds[valid_idx]) ** .5)) 
    del clf, dtrain, dvalid
    gc.collect()

print('Full rmse %.6f' % mean_squared_error(train['fare_amount'], oof_preds)**.5)
# Write submission file and plot feature importance
sub_df = test[['key']].copy()
sub_df['fare_amount'] = sub_preds
sub_df[['key', 'fare_amount']].to_csv('submission_lgb_bayesian.csv', index= False)

In [ ]:
# since distance is very important, we need to add more distance realted features
# also some time feature contribute too little, we will consider to remove them from features
# anything else: some numeric feature to categorical feature